In [ ]:
__copyright__ = "Reiner Lemoine Institut gGmbH"
__license__ = "GNU Affero General Public License Version 3 (AGPL-3.0)"
__url__ = "https://github.com/openego/eDisGo/blob/master/LICENSE"
__author__ = "mltja"

# eDisGo plot example with plotly

## Setup

#### Import packages

In [ ]:
import os
import sys
import pandas as pd
import requests
import copy

from edisgo import EDisGo
from edisgo.tools.plots import draw_plotly
from edisgo.tools.plots import dash_plot

#### Download example grid

In [ ]:
def download_ding0_example_grid():

    # create directories to save ding0 example grid into
    ding0_example_grid_path = os.path.join(
        os.path.expanduser("~"), ".edisgo", "ding0_test_network"
    )
    os.makedirs(ding0_example_grid_path, exist_ok=True)

    # download files
    filenames = [
        "buses",
        "generators",
        "lines",
        "loads",
        "network",
        "switches",
        "transformers",
        "transformers_hvmv",
    ]

    for file in filenames:
        req = requests.get(
            "https://raw.githubusercontent.com/openego/eDisGo/dev/tests/data/ding0_test_network_1/{}.csv".format(
                file
            )
        )
        filename = os.path.join(ding0_example_grid_path, "{}.csv".format(file))
        with open(filename, "wb") as fout:
            fout.write(req.content)


download_ding0_example_grid()

In [ ]:
ding0_grid = os.path.join(os.path.expanduser("~"), ".edisgo", "ding0_test_network")

#### Create edisgo objects

In [ ]:
edisgo_root = EDisGo(ding0_grid=ding0_grid)
edisgo_root.set_time_series_worst_case_analysis()
edisgo_root.analyze()

In [ ]:
edisgo_copy = copy.deepcopy(edisgo_root)

In [ ]:
edisgo_reinforced = copy.deepcopy(edisgo_root)
edisgo_reinforced.reinforce()

## Plots

### draw_plotly function

In the following different plotting options are shown.

The different options for node colors (set using parameter `mode_nodes`) and line colors (set using parameter `mode_lines`) are:

- mode_nodes
    - 'voltage_deviation' - shows the deviation of the node voltage relative to 1 p.u.
    - 'adjacencies' - shows the the number of connections of the graph
- mode_lines
    - 'relative_loading' - shows the line loading relative to the s_nom of the line
    - 'loading' - shows the loading
    - 'reinforce' - shows the reinforced lines in green


The different options for used coordinates (set using parameter `grid`) are:

- grid
    - grid object - the coordinate origin is set to the stations coordinates
    - False - the coordinates are not modified 

Hovering over nodes and lines shows some information on them.

In [ ]:
edisgo_obj = edisgo_root
grid = edisgo_obj.topology.mv_grid
G = grid.graph

In [ ]:
# plotting relative loading and voltage deviation, with grid coordinates modified to have the station in the origin
mode_lines = "relative_loading"
mode_nodes = "voltage_deviation"

fig = draw_plotly(
    edisgo_obj, G, line_color=mode_lines, node_color=mode_nodes, grid=grid
)
fig.show()

In [ ]:
# plotting loading and voltage deviation, with unchanged coordinates
mode_lines = "loading"
mode_nodes = "voltage_deviation"

fig = draw_plotly(
    edisgo_obj, G, line_color=mode_lines, node_color=mode_nodes, grid=False
)
fig.show()

In [ ]:
# plotting reinforced lines and node adjacencies
edisgo_obj = edisgo_reinforced
G = edisgo_obj.topology.mv_grid.graph

mode_lines = "reinforce"
mode_nodes = "adjacencies"

fig = draw_plotly(
    edisgo_obj, G, line_color=mode_lines, node_color=mode_nodes, grid=False
)
fig.show()

### Dash plot app which calls draw_plotly
One edisgo object creates one large plot. Two or more edisgo objects create two adjacent plots, where the objects to be plotted are selected in the dropdown menu.

#### One interactive plot

In [ ]:
app = dash_plot(edisgo_objects=edisgo_root)
app.run_server(mode="inline")

#### Two interactives plots side by side

In [ ]:
app = dash_plot(
    edisgo_objects={"edisgo_obj_1": edisgo_root, "edisgo_obj_2": edisgo_reinforced}
)
app.run_server(mode="inline")

#### Two interactives plots side by side with 3 EDisGo objects

Choose your edisgo objects in the drop down menu.

In [ ]:
app = dash_plot(
    edisgo_objects={
        "edisgo_obj_1": edisgo_root,
        "edisgo_obj_2": edisgo_reinforced,
        "edisgo_obj_3": edisgo_copy,
    }
)
app.run_server(mode="inline")